# MA6202: Laboratorio de Ciencia de Datos

**Profesor: Nicolás Caro**

**24/04/2020 - E1 S4**


**Integrantes del grupo**:  Benjamín Barrientos, Francisco Vásquez y Kurt Walsen

## Ejercicio 1

La estructura de esta evaluación consta de 7 preguntas. Se evaluá desde la pregunta 1 a la 6, la pregunta 7 es opcional. Tenga en cuenta que un problema de programación puede (por lo general) resolverse de múltiples maneras. Sin embargo, para optar al puntaje completo en cada pregunta, siga las indicaciones de los enunciados y utilice solo herramientas vistas en el curso. 

En lo que sigue de la evaluación, **no** estará permitido usar librerías ni módulos diferentes a los declarados en la siguiente celda. 

In [0]:
'''Celda de modulos obligatorios.'''

# Librerias de manejo de consultas
import requests
import json

# Stack cientifico
import numpy as np
from scipy import stats

# Librerias de manejo de tiemop
from datetime import date, datetime
from time import sleep

# Manejo de funciones
from functools import singledispatch, update_wrapper

El objetivo global de esta evaluación es medir conocimientos sobre el manejo de objetos, decoradores y librerías. 
De manera simultanea, se evaluará el manejo del sistema *Git*. Por tal motivo, para este ejercicio se exige:

1. Generar un repositorio de control de versiones *Git* donde se trabajará con este archivo.
2. Por lo menos un commit por pregunta. 
3. Por lo menos un merge. 

El formato de entrega de esta evaluación es un archivo **.zip** con el repositorio correspondiente a este trabajo. 

### Primera Parte: Construcción de consultas

Una interfaz de programación de aplicaciones, API por sus siglas en ingles, es un conjunto de rutinas, protocolos y herramientas para crear software. Básicamente, su función es especificar cómo interactúan los componentes de una aplicación. 

Existe una variedad de API's con las que se puede acceder a bases de datos, estas funcionan como una capa de software intermedia, que acepta peticiones de usuarios (consultas) y entrega información empaquetada en alguna estructura estándar (respuestas). 

Una API en particular es la de [United States Geological Survey (USGS)](https://earthquake.usgs.gov/fdsnws/event/1/). Esta API permite acceder a bases de datos contenedoras de información sobre movimientos telúricos en todo el mundo. 

El objetivo de esta primera parte, es conectarse a la API de _USGS_ para recolectar la información de tales eventos entre los años 1920 y 2020. 

Para comenzar preste atención al siguiente *decorador*.

In [0]:
def singledispatchmeth(func):
    ''' Decorador que extiende a functools.singledispatch.
    
    Se puede utilizar sobre un metodo dentro de la definicion de una clase 
    para posteriormente registrar comportamientos según tipo de dato, dentro 
    de la misma clase.
    
    Args: 
    
        func: function, class method
             Metodo a decorar
    
    Returns:
    
        class method
            Retorna un metodo del tipo singledispatch.
            
    '''
    
    despachador = singledispatch(func)

    def wrapper(*args, **kwargs):
        return despachador.dispatch(args[1].__class__)(*args, **kwargs)

    wrapper.register = despachador.register
    wrapper.registry = despachador.registry
    update_wrapper(wrapper, func)
    
    return wrapper

Como podrá ver, la función anterior permite **replicar exactamente** el comportamiento del decorador `@functools.singledispatch` **sobre métodos** dentro de la definición de una clase. 


### Pregunta 1 

Se busca hacer consultas temporales a la API de USGS, por este motivo es necesario que se extienda la clase `date` de la librería `datetime`. Un objeto tipo `date` se inicializa con 3 argumentos enteros del tipo `año`, `mes`, `dia` (en ese orden).

1. Inicialice un objeto tipo `date` para la fecha *01/01/1920*. Muestre en pantalla los atributos `.year`, `.month` y `.day`.

2. Extienda la clase `date` a una clase denotada por `DateConSuma`. Esta nueva clase debe sobrecargar el operador `+`, de manera tal, que si `data_s` es un objeto tipo `DateConSuma` y `tup = (a,b,c)` es una tupla de 3 datos tipo `int`, entonces la operación `data_s + tup` retorna un objeto tipo `DateConSuma` con atributos `.year`, `.month` y `.day` validos, dados por `data_s.year + a`,  `data_s.month + b`, `data_s.day + c`. Para esto:
    1. Decore adecuadamente el método a sobrecargar, haciendo referencia a la clase base de `DateConSuma` por medio de `super()`.
    2. Registre el comportamiento de dicho método decorado para que opere con tuplas, levante una excepción si la tupla a operar no tiene largo 3 o si alguno de sus elementos no es del tipo `int`.
    
**Obs**: El método resultante no es conmutativo, es decir, no espera que `tup + data_s` esté definido. 

In [3]:
# Respuesta P1
fecha = date(1920,1,1)
print(fecha.year, fecha.month, fecha.day)

class DateConSuma(date):
    '''Clase derivada de date.
    
    Extiende a la clase date agregando suma de date con tuplas, sobrecargando
    el operador +.
    '''
        
    @singledispatchmeth    
    def __add__(self, fecha):
        '''Método que permite la suma de objetos DateConSuma.
        
        Args
        ----
        fecha: DateConSuma
            Fecha a sumar.
        
        Returns
        -------
        DateConSuma
            Resultado de la suma con `fecha`.
        
        Raises
        ------
        TypeError
            Si `fecha` no es de la clase DateConSuma.
        
        '''
        
        if not isinstance(fecha,DateConSuma):
            raise TypeError('Objeto entregado no corresponde a DateConSuma')
        
        a, b, c = fecha.year,fecha.month,fecha.day
        a_date=DateConSuma(super().year + a, super().month + b, super().day + c)
        return a_date
    
    @__add__.register(tuple)
    def _(self,tupla): 
        '''Sobrecarga al operador + para operar DateConSuma con tuplas.
        
        Args
        ----
        tupla: tuple
            Tupla que codifica una fecha con valores (year,month,day).
            Debe ser de largo 3.
        
        Returns
        -------
        DateConSuma
            Resultado de la suma con `tupla`.
            
        Raises
        ------
        ValueError
            Si `tupla` no tiene largo 3.
            
        TypeError
            Si `tupla` tiene al menos un elemento que no sea int. 
        '''
        if len(tupla)!=3:
            raise ValueError('Tupla entregada no tiene largo 3')
        
        a,b,c= tupla
        
        if type(a) != int or type(b) != int or type(c) != int:
            raise TypeError('Uno o más elementos de la tupla no corresponden a int')
        
        a_date=DateConSuma(self.year + a,self.month + b,self.day + c)

        return a_date

1920 1 1


In [4]:
#Ejemplo:

print('Opera entre objetos DateConSuma:')
fecha_1=DateConSuma(1920,1,1)
fecha_2=DateConSuma(1925,1,1)
print('{} + {} = {}'.format(fecha_1,fecha_2,fecha_1+fecha_2))

print('Opera entre objeto DateConSuma y tuple:')
tupla = (1,1,1)
print('{} + {} = {}'.format(fecha_1,tupla,fecha_1+tupla))

Opera entre objetos DateConSuma:
1920-01-01 + 1925-01-01 = 3845-02-02
Opera entre objeto DateConSuma y tuple:
1920-01-01 + (1, 1, 1) = 1921-02-02


### Pregunta 2

Defina la clase `IteratorFechas` que retorne objetos de la clase `DateConSuma` espaciados cada $n$ años comenzando en 1920-01-01 y finalizando en 2020-01-01. Para ello:

1. Defina el atributo estático y privado `desde_fecha`, este se asocia a un objeto `DateConSuma` que representa la fecha 1920-01-01. 
    
2.  Defina el atributo estático y privado `hasta_fecha` este se asocia a un objeto `DateConSuma` que representa la fecha 2020-01-01. 
    
3. El constructor debe recibir el parámetro `n`, cuyo valor se debe almacenar en el atributo `espaciado`.
    
4. Defina el método mágico `__iter__` donde sólo se retorne el objeto instanciado.

5. Defina el método mágico `__next__`. Para esto, si `N` representa el valor del atributo `espaciado`, entonces:
    
    1. Si `desde_fecha` + `(N,0,0)` (sobrecarga de `+`) representa una fecha anterior a la de `hasta_fecha`, entonces se debe retornar una tupla con el valor de `desde_fecha` en su primera componente y de `desde_fecha` + `(N,0,0)` en su segunda componente. De igual manera, se requiere que el método `__next__` actualice el valor del atributo `desde_fecha` reasignandole el valor `desde_fecha` + `(N,0,0)`. *Hint*: Observe que `DateConSuma` hereda de `date`, por lo tanto las comparaciones del tipo `a < b` son validas para esta nueva clase. 
    
    2.  Si lo anterior no se cumple, entonces levante una excepción del tipo  `StopIteration` con el mensaje, `'fecha final excedida'`.
    
6. Inicialice el objeto `iter_fechas` de la clase `IteratorFechas` con un espaciado de 5 años. Mediante un ciclo `for` itere sobre `iter_fechas` e imprima sobre las variables iteradoras, de manera tal, que se logre un output del tipo:

```
        Desde : 1920-01-01 Hasta:  1925-01-01
        Desde : 1925-01-01 Hasta:  1930-01-01
            ...
        Desde : 2015-01-01 Hasta:  2020-01-01
```

In [0]:
# Respuesta P2
class IteratorFechas:
    '''
    Clase que retorna objetos DateConSuma espaciado n veces, empezando 1920-01-01 y terminando en 2020-01-01.
    
    Tiene asociada las propiedad n, que será el espaciado entre las fechas mencionadas (asociado a los años).
    '''
    __desde_fecha = DateConSuma(1920,1,1)
    __hasta_fecha = DateConSuma(2020,1,1)
    
    def __init__(self,n):
        self.espaciado = n
    
    def __iter__(self):
        return self
        
    def __next__(self):
        '''
        Define el método mágico __next__ que retorna tuplas.
        
        Toma el espaciado y verifica si la fecha actual no ha superado el tope, 2020-01-01. Si no, retorna la tupla 
        (year,month,day),(year + N, month, day) que son objetos DateConSuma.
                
        Returns:
            Tuple, compuesto por elementos DateConSuma.
        '''
        N=self.espaciado
        fecha_sgte = self.__desde_fecha+(N,0,0)
        
        if fecha_sgte <= self.__hasta_fecha:
            fecha_inicio = self.__desde_fecha
            self.__desde_fecha += (N,0,0)
            return (fecha_inicio,fecha_sgte)
        
        else:    
            raise StopIteration('Fecha final excedida')

In [6]:
# Vemos que el ejemplo calza con un espaciado de 5 años
iter_fechas=IteratorFechas(5)
for i,j in iter_fechas:
    print('Desde: {} Hasta: {}'.format(i,j))

Desde: 1920-01-01 Hasta: 1925-01-01
Desde: 1925-01-01 Hasta: 1930-01-01
Desde: 1930-01-01 Hasta: 1935-01-01
Desde: 1935-01-01 Hasta: 1940-01-01
Desde: 1940-01-01 Hasta: 1945-01-01
Desde: 1945-01-01 Hasta: 1950-01-01
Desde: 1950-01-01 Hasta: 1955-01-01
Desde: 1955-01-01 Hasta: 1960-01-01
Desde: 1960-01-01 Hasta: 1965-01-01
Desde: 1965-01-01 Hasta: 1970-01-01
Desde: 1970-01-01 Hasta: 1975-01-01
Desde: 1975-01-01 Hasta: 1980-01-01
Desde: 1980-01-01 Hasta: 1985-01-01
Desde: 1985-01-01 Hasta: 1990-01-01
Desde: 1990-01-01 Hasta: 1995-01-01
Desde: 1995-01-01 Hasta: 2000-01-01
Desde: 2000-01-01 Hasta: 2005-01-01
Desde: 2005-01-01 Hasta: 2010-01-01
Desde: 2010-01-01 Hasta: 2015-01-01
Desde: 2015-01-01 Hasta: 2020-01-01


La API USGS utiliza el protocolo http para recibir consultas y entregar datos de respuesta. Por lo anterior, para hacer consultas a la API, se deben producir direcciones URL siguiendo un formato especifico, en este caso el formato de consulta sigue el patrón:

        https://earthquake.usgs.gov/fdsnws/event/1/ `Metodo?Parametros`
    
En el campo `Metodo`  se utiliza `query?format=geojson` lo que implica que la respuesta entregada por la API vendrá en formato `.json`, es decir _JavaScript Object Notation_. Para efectos prácticos, este es un formato que permite almacenar información mediante una escritura análoga a los `dict` de Python. Su manejo se hace con la librería `json`. 

En cuanto al campo `Parametros`, estos están definidos en la documentación de la [API](https://earthquake.usgs.gov/fdsnws/event/1/) y permiten definir la información a consultar. Los parámetros que serán útiles para el ejercicio son:

* **Geográficos**: que servirán para definir la zona geográfica de interés mediante un rectangulo de latitudes y longitudes. Estos son: `minlatitude`, `minlongitude`, `maxlatitude` y `maxlongitude`.

* **Temporales**: que servirán para definir el periodo de consulta. Se trata de: `starttime` y `endtime`.

Por lo anterior, si se desea consultar por los eventos ocurridos entre el primero de enero de 1920 y el primero de enero de 1925 la consulta sería de la forma:

[https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=1920-01-01&endtime=1925-01-01&minlatitude=-56.72500008&minlongitude=-80&maxlatitude=-17.49839982&maxlongitude=-66.07534742](https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=1920-01-01&endtime=1925-01-01&minlatitude=-56.72500008&minlongitude=-80&maxlatitude=-17.49839982&maxlongitude=-66.07534742)

Note que los parámetros geográficos utilizados son:

```python
'minlatitude': -56.72500008, 
'minlongitude': -80,
'maxlatitude': -17.49839982,
'maxlongitude': -66.07534742,
``` 

### Pregunta 3

Dado que la API entrega un máximo de 20.000 respuestas por consulta y que Chile es un país con mucha actividad sísmica, es necesario realizar múltiples consultas para capturar **todos** los movimientos telúricos en cuestión. 

Para efectos de este ejercicio, las consultas a la API se mantendrán en una zona geográfica fija, es decir, los parámetros geográficos no varían. Se busca entonces, obtener información variando unicamente las parámetros temporales.

1. Para la clase `ConstructorConsultasUSGS` defina el método `construye_consulta` que usa los atributos privados `fixed_loc` y `base_url`. Además, tal método debe recibir los argumentos `desde_fecha` y `hasta_fecha` para retornar una consulta en string con el formato admitido por la API.  

**Ejemplo**

Se espera que el método:

```python
date_1 = DateConSuma(1920, 1, 1)
date_2 = DateConSuma(1925, 1, 1)

ConstructorConsultasUSGS.construye_consulta(date_1, date_2)
```

Retorne el string 

```
'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=1920-01-01&endtime=1925-01-01&minlatitude=-56.72500008&minlongitude=-80&maxlatitude=-17.49839982&maxlongitude=-66.07534742'
```

In [0]:
class ConstructorConsultasUSGS:
    '''
    Clase constructora de consultas a la API USGS.
    '''

    # Atributos fijos de consulta
    __fixed_loc = {
        'minlatitude': -56.72500008,
        'minlongitude': -80,
        'maxlatitude': -17.49839982,
        'maxlongitude': -66.07534742,
    }
    
    # URL Base
    __base_url = "https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson"

    def construye_consulta(self, desde_fecha, hasta_fecha):
        '''Construye el link de una consulta y retorna el URL en forma de string.
        
        Toma dos objetos DateConSuma, desde_fecha y hasta fecha y crea el URL para hacer
        la consulta entre esos años.
        Args:
            desde_fecha: Fecha de donde se quiere empezar, tipo DateConSuma.
            
            hasta_fecha: Fecha donde se quiere terminar, tipo DateConSuma.
        Returns:
            String, con la query
        '''
        query = '{}&starttime={}&endtime={}'.format(self.__base_url,desde_fecha,hasta_fecha)
        
        # Creamos la consulta con los datos fijos
        for key in self.__fixed_loc:
            query = '{}&{}={}'.format(query,key,self.__fixed_loc[key])
            
        return query

In [54]:
# Hacemos la consulta del ejemplo
desde_fecha=DateConSuma(1920,1,1)
hasta_fecha=DateConSuma(1925,1,1)
print(ConstructorConsultasUSGS().construye_consulta(desde_fecha,hasta_fecha))

https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=1920-01-01&endtime=1925-01-01&minlatitude=-56.72500008&minlongitude=-80&maxlatitude=-17.49839982&maxlongitude=-66.07534742


2. Cree la clase `GeneradorConsulta` que herede de las clases `ConstructorConsultasUSGS` e `IteratorFechas`. Esta nueva clase es un *iterator* que en cada paso entrega una consulta a la API. Para ello haga _overridding_ (o anulación) de los metodos:

    1. `__init__`: haga uso de la función `super` en dos ocasiones. *Hint*: Recuerde el MRO *como cadena* y su interacción con `super`, incialice clase base.
    2. `__next__`: haga uso de la función `super` en dos ocasiones. *Hint*: Debe retornar una url construida con una *tupla* de fechas. 
    
**Ejemplo**

Las primera y última iteración deben retornar:

```
'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=1920-01-01&endtime=1925-01-01&minlatitude=-56.72500008&minlongitude=-80&maxlatitude=-17.49839982&maxlongitude=-66.07534742'
   ...
'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2015-01-01&endtime=2020-01-01&minlatitude=-56.72500008&minlongitude=-80&maxlatitude=-17.49839982&maxlongitude=-66.07534742'
```

In [0]:
# Respuesta P3
class GeneradorConsulta(ConstructorConsultasUSGS,IteratorFechas):
    '''
    Clase iterator que genera Consultas por medio de clases heradadas.
    '''
    def __init__(self):
        init_1 = super().__init__(5)
        init_2 = super(ConstructorConsultasUSGS).__init__()
         
    def __next__(self):
        '''
        Define el método mágico __next__ que retorna tuplas.
        
        Usa las clases ConstructorConsultaUSGS y Iterator Fechas para hacer una iteración con las consultas.
        Returns:
            str, que representa las consulta.
        '''
        fecha_inicio, fecha_sgte = super().__next__()
        query = super().construye_consulta(fecha_inicio, fecha_sgte)
        return query

In [10]:
# Ejemplo de las queries
print('Ejemplo de queries entre 1920-01-01 y 1925-01-01')
for query in GeneradorConsulta():
    print(query)

Ejemplo de queries entre 1920-01-01 y 1925-01-01
https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=1920-01-01&endtime=1925-01-01&minlatitude=-56.72500008&minlongitude=-80&maxlatitude=-17.49839982&maxlongitude=-66.07534742
https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=1925-01-01&endtime=1930-01-01&minlatitude=-56.72500008&minlongitude=-80&maxlatitude=-17.49839982&maxlongitude=-66.07534742
https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=1930-01-01&endtime=1935-01-01&minlatitude=-56.72500008&minlongitude=-80&maxlatitude=-17.49839982&maxlongitude=-66.07534742
https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=1935-01-01&endtime=1940-01-01&minlatitude=-56.72500008&minlongitude=-80&maxlatitude=-17.49839982&maxlongitude=-66.07534742
https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=1940-01-01&endtime=1945-01-01&minlatitude=-56.72500008&minlongitude=-80&maxlatitude=-17.4

## Segunda Parte: Recolección

Construida la clase que genera las diferentes consultas, se procede a la recolección y analisis. Los datos seran almacendos en un diccionario con la siguiente estructura:

```python
{
    'id': [id_1, id_2, ..., id_n],
    'lugar': [lugar_1, lugar_2, ..., lugar_n],
    'timestamp': [timestamp_1, timestamp_2, ..., timestamp_n],
    'magnitud': [magnitud_1, magnitud_2, ..., magnitud_n],
    'longitud': [longitud_1, longitud_2, ..., longitud_n ],
    'latitud': [latitud_1, latitud_2, ..., latitud_n]
}
```

### Pregunta 4

Observe que la estructura anterior corresponde a un diccionario de listas. Se busca facilitar el almacenamiento de la información recolectada mediante una concatenación especial de diccionarios. Con tal fin, extienda la clase `dict`, definiendo la clase `DictDeListas`. Para esta nueva clase, defina el método `extiende_listas`, este método: 

1. Recibe como argumento otro objeto del tipo `DictDeListas` y levanta una excepción del tipo `ValueError` con el mensaje  `'El argumento debe ser de la clase DictDeListas'` en caso contrario.
2. Para cada llave del objeto instanciado, concatena las listas asociadas a esa llave en cada uno de los dos objetos involucrados (el instanciado y el recibido como argumento). Haga uso del método `items` e itere sobre el segundo objeto `DictDeListas`.
   
**Ejemplo**

El código

```python
# Se generan objetos DictDeListas con llaves en comun 
dict_ls_1 = DictDeListas({'llave': [1, 2,3]})
dict_ls_2 = DictDeListas({'llave': [4, 5, 200]})

# Se utiliza el metodo y se muestra su representacion __repr__
dict_ls_1.extiende_listas(dict_ls_2)
dict_ls_1
```

Genera el output:

```
{'llave': [1, 2, 3, 4, 5, 200]}
```

In [0]:
# Respuesta P4
class DictDeListas(dict):
    '''Clase derivada de dict. 
    
    Facilita el almacenamiento de la información recolectada
    mediante una concatenación especial de diccionarios.
    
    '''
    def extiende_listas(self, dict_1):
        '''Concatena los elementos de las llaves de cada DictDeListas.
        
        Permite concatenar los elementos pertenecientes a las
        llaves de cada diccionario y almacenar la concatenación
        en la llave respectiva de cada uno.
        
        Args
        ----
        dict_1: DictDeListas
        
        Returns
        -------
            None
            
        Raises
        ------
        TypeError
            Si `dict_1` no es de la clase DictDeListas.
        '''
        
        if not isinstance(dict_1, DictDeListas):
            raise TypeError('El argumento debe ser de la clase DictDeListas')

        for key,lista in dict_1.items():
            self[key],dict_1[key] = self[key]+lista, lista+self[key]

In [12]:
# Se generan objetos DictDeListas con llaves en comun 
dict_ls_1 = DictDeListas({'llave': [1, 2,3]})
dict_ls_2 = DictDeListas({'llave': [4, 5, 200]})

# Se utiliza el metodo y se muestra su representacion __repr__
dict_ls_1.extiende_listas(dict_ls_2)
dict_ls_1

{'llave': [1, 2, 3, 4, 5, 200]}

Cuando se tiene una URL, basta introducirla al navegador para obtener su información asociada. En Python esto se puede realizar con la librería `requests`. En particular, el método `get` de tal librería, permite efectuar consultas en la API USGS, en efecto, si se analiza la respuesta de:

In [0]:
resp = requests.get(
    'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=1920-01-01&endtime=1925-01-01&minlatitude=-56.72500008&minlongitude=-80&maxlatitude=-17.49839982&maxlongitude=-66.07534742'
)

Notamos que dicho objeto posee las siguientes caracteristicas:

1. `__enter__` y `__exit__`. 
2. `json`, el cual retorna el contenido codificado de la respuesta. 
3. Un atributo `.ok` que retorna `True` si la consulta se responde con exito.

Se puede observar el contenido `json` de la respuesta obtenida por medio de:

In [14]:
display(resp.json())
resp.close()

{'bbox': [-73.182, -38.708, 15, -67.848, -17.734, 70],
 'features': [{'geometry': {'coordinates': [-70.755, -28.222, 45],
    'type': 'Point'},
   'id': 'iscgem910820',
   'properties': {'alert': None,
    'cdi': None,
    'code': '910820',
    'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=iscgem910820&format=geojson',
    'dmin': None,
    'felt': None,
    'gap': None,
    'ids': ',iscgem910820,',
    'mag': 6.7,
    'magType': 'mw',
    'mmi': 6.892,
    'net': 'iscgem',
    'nst': None,
    'place': 'Atacama, Chile',
    'rms': None,
    'sig': 691,
    'sources': ',iscgem,',
    'status': 'automatic',
    'time': -1449266702000,
    'title': 'M 6.7 - Atacama, Chile',
    'tsunami': 0,
    'type': 'earthquake',
    'types': ',origin,shakemap,',
    'tz': None,
    'updated': 1585298188991,
    'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/iscgem910820'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-73.182, -38.708, 35], 'type': 'Point'},

Aqui, se aprecia que la respuesta es un `dict` que contiene diferentes pares de (llaves, valor) de tipo (`str`, `obj`) donde`obj` puede ser `dict`, `list`, `str`, `int`, `float`, `NoneType`, etc...

Para esta pregunta, nos centraremos en la llave `'features'`, que contiene una lista con elementos de tipo `dict` con la información recolectada. 

### Pregunta 5

Las siguientes funciones lambda retornan los atributos de interés, observe que está en presencia de un diccionario de funciones

In [0]:
funciones_dict = {
    'id': lambda x: x['id'],
    'lugar': lambda x: x['properties']['place'],
    'timestamp': lambda x: datetime.fromtimestamp(x['properties']['time'] // 1000),
    'magnitud': lambda x: x['properties']['mag'],
    'longitud': lambda x: x['geometry']['coordinates'][0],
    'latitud': lambda x: x['geometry']['coordinates'][1]
}

Así para obtener los atributos de interés del primer elemento asociado a `'features'` (primera observación) se puede ejecutar:

In [16]:
{k: func(resp.json()['features'][0]) for k, func in funciones_dict.items()}

{'id': 'iscgem910820',
 'latitud': -28.222,
 'longitud': -70.755,
 'lugar': 'Atacama, Chile',
 'magnitud': 6.7,
 'timestamp': datetime.datetime(1924, 1, 29, 1, 54, 58)}

Recolecte los datos de la API haciendo uso de `funciones_dict`. En el siguiente procedimiento complete las 3 lineas indicadas con:
```python
    _ _ _ _ # Completar {n}

```
Para esto:
1. En **Completar 1** haga uso de la función `map` y una conversión de tipo de dato a `list`.
2. En **Completar 2** y **Completar 3**, opere sobre `observaciones_dict` y obtenga un objeto `DictDeListas` con la concatenación de todos los atributos de interés, para cada las consulta realizada a la API. *Hint* : recuerde la razón por la que definimos `DictDeListas`.

**Obs**: La cantidad de observaciones recolectadas debería ser mayor a  63.500. Puede acceder a este dato por medio de `len(observaciones_dict['id'])`.

In [17]:
# Respeusta P5

# Inicializar el contenedor de los datos recolectados
observaciones_dict = DictDeListas()

# Generar las url de las consultas a la API
for consulta_str in GeneradorConsulta():
    
    # Abre un context manager
    with requests.get(consulta_str) as respuesta:
        
        # Verifica con el atributo .ok si se recibe la informacion
        assert respuesta.ok, 'Respuesta erronea'
    
        # Almacena las features de interes
        respuesta_dict = respuesta.json()
        respuesta_features_lista = respuesta_dict['features']

        # Reporta numero de resultados obtenidos en la iteracion
        print('Se obtuvieron', len(respuesta_dict["features"]), 'observaciones \n')
        
        # Inicializa un contenedor de resultados
        iter_dict = DictDeListas()

        for k, func in funciones_dict.items():

            iter_dict[k] = list(map(func,respuesta_features_lista)) # Completar 1
        
        # Verifica si observaciones_dict esta vacio
        if observaciones_dict:

            # Colapsamos los resultados
            observaciones_dict.extiende_listas(iter_dict)

        else:
            # Si es vacio, lo creamos
            observaciones_dict = iter_dict # Completar 3

        sleep(5) # espera 5 segundos antes de realizar la próxima consulta

Se obtuvieron 11 observaciones 

Se obtuvieron 10 observaciones 

Se obtuvieron 6 observaciones 

Se obtuvieron 5 observaciones 

Se obtuvieron 10 observaciones 

Se obtuvieron 5 observaciones 

Se obtuvieron 18 observaciones 

Se obtuvieron 24 observaciones 

Se obtuvieron 75 observaciones 

Se obtuvieron 49 observaciones 

Se obtuvieron 610 observaciones 

Se obtuvieron 1605 observaciones 

Se obtuvieron 3604 observaciones 

Se obtuvieron 5553 observaciones 

Se obtuvieron 7810 observaciones 

Se obtuvieron 10502 observaciones 

Se obtuvieron 13248 observaciones 

Se obtuvieron 9509 observaciones 

Se obtuvieron 5553 observaciones 

Se obtuvieron 5356 observaciones 



Finalmente es posible proceder al análisis de los datos. Para ello se hará uso de  las librerías `numpy` y `scipy`. El primer paso será transformar las listas en numpy array usando los siguientes tipos de datos:

In [18]:
dtype_dict = {
    'id': np.dtype('<U64'),
    'lugar': np.dtype('<U64'),
    'timestamp': np.dtype('<M8[ns]'),
    'magnitud': np.dtype(float),
    'longitud': np.dtype(float),
    'latitud': np.dtype(float)
}
dtype_dict

{'id': dtype('<U64'),
 'latitud': dtype('float64'),
 'longitud': dtype('float64'),
 'lugar': dtype('<U64'),
 'magnitud': dtype('float64'),
 'timestamp': dtype('<M8[ns]')}

### Pregunta 6

Transforme las listas de los atributos de interés contenidas en `observaciones_dict` en un `numpy.ndarray` de `dtype` específico.  Para ello, utilice `dtype_dict`, a modo de ejemplo, para los datos contenidos en `observaciones_dict['magnitud']` se debe transformar según  `dtype_dict['magnitud']` en un `numpy.ndarray` de `dtype` `np.dtype(float)`. 
1. Utilice comprensión de diccionarios y el método `.items()` para transformar los datos. Guarde el resultado en la variable `observaciones_np_dict`. 
2. Compruebe que el evento más antiguo contenido en `observaciones_np_dict` corresponde al 3 de agosto de 1920.

In [19]:
# Respuesta P6 1

observaciones_np_dict = {k:np.array(vec, dtype =dtype_dict[k] )  for k,vec in observaciones_dict.items()}
observaciones_np_dict

{'id': array(['iscgem910820', 'iscgem911723', 'iscgemsup911356', ...,
        'usc000tb7u', 'usc000tb79', 'usc000tb6x'], dtype='<U64'),
 'latitud': array([-28.222 , -38.708 , -28.928 , ..., -31.901 , -19.9448, -31.072 ]),
 'longitud': array([-70.755 , -73.182 , -71.332 , ..., -69.87  , -70.9885, -71.347 ]),
 'lugar': array(['Atacama, Chile', 'Araucania, Chile', 'Atacama, Chile', ...,
        '76km SW of Calingasta, Argentina', '93km WNW of Iquique, Chile',
        '54km SSW of Ovalle, Chile'], dtype='<U64'),
 'magnitud': array([6.7, 6.2, 6.5, ..., 4.1, 4.2, 4.5]),
 'timestamp': array(['1924-01-29T01:54:58.000000000', '1923-11-06T17:15:23.000000000',
        '1923-05-04T22:26:50.000000000', ...,
        '2015-01-02T02:29:36.000000000', '2015-01-01T23:39:47.000000000',
        '2015-01-01T21:08:22.000000000'], dtype='datetime64[ns]')}

In [20]:
# Respuesta P6 2

# Buscamos el menor valor
valor = np.where(observaciones_np_dict['timestamp'] == np.amin(observaciones_np_dict['timestamp'])) 

# Evaluamos
observaciones_np_dict['timestamp'][valor]

array(['1920-08-03T19:57:21.000000000'], dtype='datetime64[ns]')

### Pregunta 7 (opcional)

El objetivo del ejercicio es obtener estadísticas descriptivas de los movimientos telúricos ocurridos en Chile en los últimos 100 años. Para mostrar dichas estadísticas definiremos la función `muestra_estadisticas_descriptivas` que toma como argumento un  `numpy.ndarray` y muestra en pantalla las siguientes estadísticas según la clase de los elementos contenidos en dicho arreglo:
- `'np.float64'` y `numpy.datetime64` (`datetime` en nanosegundos): número de observaciones, promedio, mediana, primer cuartil, tercer cuartil, minimo y maximo.
- `'<U64'` (`str` de largo máximo 64): número de observaciones y moda.

Como el comportamiento debe variar según el tipo de dato del argumento, sería conveniente usar el decorador `singledispatch`. Sin embargo al verificar:
```python
>>>[val.__class__ for val in observaciones_np_dict.values()]
[numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray]
```
observamos que el todos los arrays contenidos en `observaciones_np_dict` pertenecen a la misma clase, `numpy.ndarray`. Por lo tanto es necesario volver a modificar el comportamiento de `singledispatch` para que sea capaz de distinguir entre el `dtype` del array que recibe como argumento.

1. Complete el código de la siguiente función para extender la funcionalidad de `singledispatch` para distinguir `dtype` del `numpy.ndarray` usado como argumento.  

*Hint* : Revise el código de `singledispatchmeth`, [esta página](https://stackoverflow.com/questions/24601722/how-can-i-use-functools-singledispatch-with-instance-methods) puede ser de ayuda.

In [21]:
# Respuesta P7 1
def numpydispatch(func):
    despachador = singledispatch(func)

    def wrapper(*args, **kw):
        return despachador.dispatch(_ _ _ _ )(_ _ _ _) # Completar
    
    wrapper.register = despachador.register
    update_wrapper(wrapper, func)
    return wrapper

SyntaxError: ignored

2. Para concluir complete las funciones de `muestra_estadisticas_descriptivas` asociadas a arreglos que contienen elementos de clase `np.float64` y `np.str_`. Estas deben mostrar en pantalla:
    - `'np.float64'`: número de observaciones, promedio, mediana, primer cuartil, tercer cuartil, minimo y maximo.
    - `'<U64'` (`str` de largo máximo 64): número de observaciones y moda. 
    
   *Hint*: tenga especial cuidado con los valores faltantes `np.nan` que pueden presentarse en los arreglos con elementos de clase `'np.float64'`. Puede ser útil usar las funciones de `numpy` que permiten el manejo de valores faltantes. Por ejemplo `np.nanmean` como análoga a `np.mean`. 
    
   **Obs**: se provee la función asociada a los array con elementos de la clase `np.datetime64` pues el método `view` está fuera del alcance del ejercicio.
   
   **Obs**: la notación `f"texto{var}"` es equivalente a `"texto {}".format(var)`. El cuál es un método diseñado para insertar valores dinámicos (variables del entorno) sobre un string. 

In [0]:
# Respuesta P7 2
@numpydispatch
def muestra_estadisticas_descriptivas(array):
    '''Define el comportamiento para objetos en general fuera de registry'''
    print(f"numero de observaciones: {len(array):,}")
        
@muestra_estadisticas_descriptivas.register(np.datetime64)
def _(array):
    '''Define el comportamiento para los objetos '''
    
    # transforma array para que puedan calcularse las estadisticas deseadas
    trans_array = observaciones_arr_dict['timestamp'].view('i8')
    
    # inicializa el contenedor de los str a mostrar en pantalla
    lista_str = []
    
    # genera los str a mostrar en pantalla
    lista_str.append(f"numero de observaciones: {len(array):,}")
    lista_str.append(
        f"media: {trans_array.mean().astype('datetime64[ns]')}"
    )
    lista_str.append(
        f"mediana: {np.median(trans_array).astype('datetime64[ns]')}"
    )
    lista_str.append(
        f"primer cuartil: {np.quantile(trans_array, .25).astype('datetime64[ns]')}"
    )
    lista_str.append(
        f"tercer cuartil: {np.quantile(trans_array, .75).astype('datetime64[ns]')}"
    )
    lista_str.append(
        f"minimo: {trans_array.min().astype('datetime64[ns]')}"
    )
    lista_str.append(
        f"maximo: {trans_array.max().astype('datetime64[ns]')}"
    )
    
    # muestra los elementos de la lista separados por un salto de linea
    print('\n'.join(lista_str))
    
@muestra_estadisticas_descriptivas.register(np.float64)
def _(array):
    # Completar
    
    
@muestra_estadisticas_descriptivas.register(np.str_)
def _(array):
    # Completar